In [90]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-symptoms-checker/Cleaned-Data.csv
/kaggle/input/covid19-symptoms-checker/Raw-Data.csv


In [91]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier , plot_tree
from sklearn.metrics import r2_score


In [ ]:
df=pd.read_csv(r'/kaggle/input/covid19-symptoms-checker/Cleaned-Data.csv')

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder=OrdinalEncoder()
category=['Country']

feature=df[category]
encoder.fit(feature)

df[category] = pd.DataFrame(encoder.transform(feature))

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated()

In [ ]:
df.describe()

In [ ]:
df['target']=df['Severity_Severe']

In [ ]:
sns.boxplot(x=df['target'])
plt.title("Boxplot for target")
plt.show()

In [ ]:
Q1 = df['target'].quantile(0.25)
Q3 = df['target'].quantile(0.75)


IQR = Q3-Q1
IQR

outliers = df[(df['target']<Q1 - 1.5 * IQR)|(df['target'] >Q3+1.5 * IQR)]

In [ ]:
df_clean = df[(df['target']>=Q1 - 1.5IQR)&(df['target'] <=Q3+1.5IQR)]

In [ ]:
sns.boxplot(x = df_clean['target'])
plt.title('Box plot wz out outliers')
plt.show()

In [ ]:
# To make the report more organized

def train_model(model, x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    test_pred = model.predict(x_test)
    train_pred = model.predict(x_train)

    print('Classification Report for TRAIN DATA')
    print('--'*10)
    print(classification_report(y_train,train_pred))
    print('=='*20)
    print('Classification Report for TEST DATA')
    print('--'*10)
    print(classification_report(y_test,test_pred))

    cm_train = confusion_matrix(y_train,train_pred)
    cm_test = confusion_matrix(y_test,test_pred)
    return cm_train,cm_test

In [ ]:
x=df.drop(['target', 'Severity_Severe', 'Severity_Mild', 'Severity_Moderate', 'Severity_None'], axis=1)
y=df['target']


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
model=KNeighborsClassifier()
model.fit(x_train,y_train)

In [ ]:
train_model(model, x_train,x_test,y_train,y_test)

In [ ]:
T_mod=DecisionTreeClassifier(criterion='entropy',max_depth=3)
T_mod.fit(x_train,y_train)

In [ ]:
plt.figure(figsize=(20,10)) 
tree = plot_tree(T_mod, filled=True, feature_names=df.columns)

plt.show()

In [ ]:
train_model(T_mod, x_train,x_test,y_train,y_test)

In [ ]:
rf = RandomForestClassifier(n_estimators=2, random_state=7)
rf.fit(x_train,y_train)

In [ ]:
rf_pred_train = rf.predict(x_train)
rf_pred_test = rf.predict(x_test)

In [ ]:
print(accuracy_score(rf_pred_train,y_train))
print(accuracy_score(rf_pred_test,y_test))